### Rag with typesense

In [ ]:
import typesense

In [ ]:
client = typesense.Client({
    'nodes': [{
    'host': 'wr8f1hjzig39xmqcp-1.a1.typesense.net',  # For Typesense Cloud use xxx.a1.typesense.net
    'port': '443',       # For Typesense Cloud use 443
    'protocol': 'https'    # For Typesense Cloud use https
  }],
  'api_key':'OsCOsSOA8bIS9Hj3xS9BgOrGgPFI8ay3',
  'connection_timeout_seconds': 2
})

books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string'},
    {'name': 'authors', 'type': 'string[]', 'facet': True},
    {'name': 'publication_year', 'type': 'int32', 'facet': True},
    {'name': 'ratings_count', 'type': 'int32'},
    {'name': 'average_rating', 'type': 'float'}
  ],
  'default_sorting_field': 'ratings_count'
}




In [ ]:
client

In [ ]:
with open('books.jsonl', 'r', encoding='utf-8') as jsonl_file:
    data = jsonl_file.read()
    client.collections['books'].documents.import_(data)

In [ ]:
search_parameters={
    'q':"harry potter",
    'query_by':"title,authors",
    'sort_by':"ratings_count:desc"
}

client.collections['books'].documents.search(search_parameters)

In [ ]:
search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'filter_by' : 'publication_year:<1998',
  'sort_by'   : 'publication_year:desc'
}

client.collections['books'].documents.search(search_parameters)

In [ ]:
search_parameters = {
  'q'         : 'experyment',
  'query_by'  : 'title',
  'facet_by'  : 'authors',
  'sort_by'   : 'average_rating:desc'
}

client.collections['books'].documents.search(search_parameters)

In [ ]:
### Langchain + Typsense + Groq LLM + RAG Application

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [ ]:
import os
os.environ["GROQ_API_KEY"] = ".env.GROQ_API_KEY"

In [4]:
loader = TextLoader("test.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

C:\Users\anskh\AppData\Local\Temp\ipykernel_12400\13807947.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\anskh\AppData\Local\Temp\ipykernel_12400\13807947.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [5]:
docsearch=Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params={
        "host": "wr8f1hjzig39xmqcp-1.a1.typesense.net",  # Use xxx.a1.typesense.net for Typesense Cloud
        "port": "443",  # Use 443 for Typesense Cloud
        "protocol": "https",  # Use https for Typesense Cloud
        "typesense_api_key":"OsCOsSOA8bIS9Hj3xS9BgOrGgPFI8ay3",
        "typesense_collection_name": "lang-chain"
    },
    
)

In [6]:
query = "What is artificial intelligence"
found_docs = docsearch.similarity_search(query)
print(found_docs[0].page_content)

ÿþA r t i f i c i a l   I n t e l l i g e n c e   ( A I )   i s   o n e   o f   t h e   m o s t   t r a n s f o r m a t i v e   t e c h n o l o g i e s   o f   t h e   2 1 s t   c e n t u r y ,   r e s h a p i n g   i n d u s t r i e s ,   s o c i e t i e s ,   a n d   h u m a n   i n t e r a c t i o n s .   I t   r e f e r s   t o   t h e   s i m u l a t i o n   o f   h u m a n   i n t e l l i g e n c e   i n   m a c h i n e s   t h a t   a r e   d e s i g n e d   t o   t h i n k ,   l e a r n ,   a n d   m a k e   d e c i s i o n s .   B y   a n a l y z i n g   v a s t   a m o u n t s   o f   d a t a   a n d   i d e n t i f y i n g   p a t t e r n s ,   A I   s y s t e m s   c a n   p e r f o r m   t a s k s   t h a t   t r a d i t i o n a l l y   r e q u i r e d   h u m a n   i n t e l l i g e n c e ,   s u c h   a s   r e a s o n i n g ,   p r o b l e m - s o l v i n g ,   a n d   l a n g u a g e   u n d e r s t a n d i n g . 
 
 
 
 E v o l u t i o n   o f   A I 
 
 T h e   c o n 

In [7]:
### Retriever
retriever = docsearch.as_retriever()
retriever

VectorStoreRetriever(tags=['Typesense', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.typesense.Typesense object at 0x000001753EECDD30>, search_kwargs={})